## PROMPT
Generate a pipeline for Databricks:

Get Data information from two web addresses:

Space launches: https://api.spacexdata.com/v3/launches

Space rockets: https://api.spacexdata.com/v3/rockets

Combine each launch record with the name of the corresponding rocket.

Send the final result (launches with rocket names) to the web address: https://httpbin.org/post

The script must provide status updates on its progress, report any errors encountered, confirm the outcome of the final data sending step, and measure/report execution times.

In [0]:
# Databricks SpaceX Data Processing Pipeline
# This pipeline fetches SpaceX launch and rocket data, joins them, and posts results to an API endpoint.

import requests
import json
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, from_json, explode
from pyspark.sql.types import StringType, StructType, StructField, ArrayType, MapType, IntegerType, BooleanType

# Initialize SparkSession
spark = SparkSession.builder.appName("SpaceX Data Pipeline").getOrCreate()

# Initialize logging
log_data = []

def log_message(message, level="INFO"):
    """Log messages with timestamp and level"""
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    log_entry = f"[{timestamp}] [{level}] {message}"
    print(log_entry)
    log_data.append(log_entry)

def measure_execution_time(func):
    """Decorator to measure execution time of functions"""
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        log_message(f"Execution of {func.__name__} took {execution_time:.2f} seconds")
        return result
    return wrapper

@measure_execution_time
def fetch_data(url, data_name):
    """Fetch data from URL and return as JSON"""
    try:
        log_message(f"Fetching {data_name} data from {url}")
        response = requests.get(url)
        response.raise_for_status()  # Raise exception for bad status codes
        data = response.json()
        log_message(f"Successfully fetched {len(data)} {data_name} records")
        return data
    except requests.exceptions.RequestException as e:
        log_message(f"Error fetching {data_name} data: {str(e)}", "ERROR")
        raise

@measure_execution_time
def convert_to_dataframe(data, data_name):
    """Convert JSON data to Spark DataFrame"""
    try:
        log_message(f"Converting {data_name} data to DataFrame")
        df = spark.createDataFrame(data)
        log_message(f"Successfully created DataFrame with {df.count()} rows and {len(df.columns)} columns")
        return df
    except Exception as e:
        log_message(f"Error converting {data_name} data to DataFrame: {str(e)}", "ERROR")
        raise

@measure_execution_time
def join_data(launches_df, rockets_df):
    """Join launches with corresponding rocket data"""
    try:
        log_message("Joining launch data with rocket names")
        
        # Select only needed columns from rockets dataframe
        rockets_slim = rockets_df.select(col("rocket_id"), col("rocket_name"))
        
        # Join launches with rockets on rocket_id
        joined_df = launches_df.join(rockets_slim, launches_df.rocket.rocket_id == rockets_slim.rocket_id, "left")
        
        # Log success details
        row_count = joined_df.count()
        log_message(f"Successfully joined data, resulting in {row_count} records")
        
        return joined_df
    except Exception as e:
        log_message(f"Error joining data: {str(e)}", "ERROR")
        raise

@measure_execution_time
def send_data_to_endpoint(data, url):
    """Send processed data to the specified endpoint"""
    try:
        log_message(f"Sending data to {url}")
        
        # Convert DataFrame to JSON format
        json_data = data.toJSON().collect()
        payload = {"data": json_data}
        
        # Send POST request to endpoint
        response = requests.post(url, json=payload)
        response.raise_for_status()
        
        # Log response details
        log_message(f"Successfully sent data. Response status: {response.status_code}")
        log_message(f"Response content: {response.text[:200]}...")
        
        return response
    except requests.exceptions.RequestException as e:
        log_message(f"Error sending data: {str(e)}", "ERROR")
        raise

def main():
    """Main pipeline function"""
    total_start_time = time.time()
    
    log_message("Starting SpaceX data pipeline")
    
    try:
        # 1. Fetch data from APIs
        launches_data = fetch_data("https://api.spacexdata.com/v3/launches", "launches")
        rockets_data = fetch_data("https://api.spacexdata.com/v3/rockets", "rockets")
        
        # 2. Convert to DataFrames
        launches_df = convert_to_dataframe(launches_data, "launches")
        rockets_df = convert_to_dataframe(rockets_data, "rockets")
        
        # 3. Join data
        result_df = join_data(launches_df, rockets_df)
        
        # 4. Show sample of processed data
        log_message("Sample of processed data:")
        result_df.select("flight_number", "mission_name", "rocket_name").show(5)
        
        # 5. Send data to endpoint
        response = send_data_to_endpoint(result_df, "https://httpbin.org/post")
        
        # Pipeline completed successfully
        total_execution_time = time.time() - total_start_time
        log_message(f"Pipeline completed successfully in {total_execution_time:.2f} seconds")
        
    except Exception as e:
        log_message(f"Pipeline failed: {str(e)}", "ERROR")
        raise
    finally:
        # Save logs to file
        log_df = spark.createDataFrame([(log,) for log in log_data], ["log_message"])
        log_df.write.mode("overwrite").csv("/dbfs/pipeline_logs/spacex_pipeline_logs.csv")

if __name__ == "__main__":
    main()

First error for claude try to give back the output

In [0]:
# Databricks SpaceX Data Processing Pipeline
# This pipeline fetches SpaceX launch and rocket data, joins them, and posts results to an API endpoint.

import requests
import json
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, from_json, explode
from pyspark.sql.types import StringType, StructType, StructField, ArrayType, MapType, IntegerType, BooleanType, FloatType, TimestampType, DateType, LongType

# Initialize SparkSession
spark = SparkSession.builder.appName("SpaceX Data Pipeline").getOrCreate()

# Initialize logging
log_data = []

def log_message(message, level="INFO"):
    """Log messages with timestamp and level"""
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    log_entry = f"[{timestamp}] [{level}] {message}"
    print(log_entry)
    log_data.append(log_entry)

def measure_execution_time(func):
    """Decorator to measure execution time of functions"""
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        log_message(f"Execution of {func.__name__} took {execution_time:.2f} seconds")
        return result
    return wrapper

@measure_execution_time
def fetch_data(url, data_name):
    """Fetch data from URL and return as JSON"""
    try:
        log_message(f"Fetching {data_name} data from {url}")
        response = requests.get(url)
        response.raise_for_status()  # Raise exception for bad status codes
        data = response.json()
        log_message(f"Successfully fetched {len(data)} {data_name} records")
        return data
    except requests.exceptions.RequestException as e:
        log_message(f"Error fetching {data_name} data: {str(e)}", "ERROR")
        raise

@measure_execution_time
def convert_to_dataframe(data, data_name):
    """Convert JSON data to Spark DataFrame with explicit schema handling"""
    try:
        log_message(f"Converting {data_name} data to DataFrame")
        
        if data_name == "launches":
            # Define schema for nested fields in launches data
            # Create a Spark DataFrame from the JSON string representation
            json_data = json.dumps(data)
            df = spark.read.json(spark.sparkContext.parallelize([json_data]))
            
            # Alternative approach with explicit schema if needed
            # df = spark.createDataFrame(json.loads(json_data))
            
        else:  # rockets data has simpler structure
            df = spark.createDataFrame(data)
            
        log_message(f"Successfully created DataFrame with {df.count()} rows and {len(df.columns)} columns")
        return df
    except Exception as e:
        log_message(f"Error converting {data_name} data to DataFrame: {str(e)}", "ERROR")
        raise

@measure_execution_time
def join_data(launches_df, rockets_df):
    """Join launches with corresponding rocket data"""
    try:
        log_message("Joining launch data with rocket names")
        
        # Select only needed columns from rockets dataframe
        rockets_slim = rockets_df.select(col("rocket_id"), col("rocket_name"))
        
        # Extract rocket_id from the rocket field in launches_df
        # The rocket field contains a struct with rocket_id
        launches_with_rocket_id = launches_df.select(
            "*", 
            col("rocket.rocket_id").alias("launch_rocket_id")
        )
        
        # Join launches with rockets on rocket_id
        joined_df = launches_with_rocket_id.join(
            rockets_slim, 
            launches_with_rocket_id.launch_rocket_id == rockets_slim.rocket_id, 
            "left"
        )
        
        # Log success details
        row_count = joined_df.count()
        log_message(f"Successfully joined data, resulting in {row_count} records")
        
        return joined_df
    except Exception as e:
        log_message(f"Error joining data: {str(e)}", "ERROR")
        raise

@measure_execution_time
def send_data_to_endpoint(data, url):
    """Send processed data to the specified endpoint"""
    try:
        log_message(f"Sending data to {url}")
        
        # Convert DataFrame to JSON format
        # Using collect to get all data to driver node - be careful with large datasets
        json_data = data.toJSON().collect()
        payload = {"data": json_data}
        
        # Send POST request to endpoint
        response = requests.post(url, json=payload)
        response.raise_for_status()
        
        # Log response details
        log_message(f"Successfully sent data. Response status: {response.status_code}")
        log_message(f"Response content: {response.text[:200]}...")
        
        return response
    except requests.exceptions.RequestException as e:
        log_message(f"Error sending data: {str(e)}", "ERROR")
        raise

def main():
    """Main pipeline function"""
    total_start_time = time.time()
    
    log_message("Starting SpaceX data pipeline")
    
    try:
        # 1. Fetch data from APIs
        launches_data = fetch_data("https://api.spacexdata.com/v3/launches", "launches")
        rockets_data = fetch_data("https://api.spacexdata.com/v3/rockets", "rockets")
        
        # 2. Convert to DataFrames
        launches_df = convert_to_dataframe(launches_data, "launches")
        rockets_df = convert_to_dataframe(rockets_data, "rockets")
        
        # 3. Print schema to help debugging
        log_message("Launches DataFrame Schema:")
        launches_df.printSchema()
        
        log_message("Rockets DataFrame Schema:")
        rockets_df.printSchema()
        
        # 4. Join data
        result_df = join_data(launches_df, rockets_df)
        
        # 5. Show sample of processed data
        log_message("Sample of processed data:")
        result_df.select("flight_number", "mission_name", "rocket_name").show(5)
        
        # 6. Send data to endpoint
        response = send_data_to_endpoint(result_df, "https://httpbin.org/post")
        
        # Pipeline completed successfully
        total_execution_time = time.time() - total_start_time
        log_message(f"Pipeline completed successfully in {total_execution_time:.2f} seconds")
        
    except Exception as e:
        log_message(f"Pipeline failed: {str(e)}", "ERROR")
        raise
    finally:
        # Save logs to file
        log_df = spark.createDataFrame([(log,) for log in log_data], ["log_message"])
        log_df.write.mode("overwrite").csv("/dbfs/pipeline_logs/spacex_pipeline_logs.csv")

if __name__ == "__main__":
    main()

another error

In [0]:
# Databricks SpaceX Data Processing Pipeline
# This pipeline fetches SpaceX launch and rocket data, joins them, and posts results to an API endpoint.
# Compatible with Spark Connect / Databricks Serverless

import requests
import json
import time
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr
from pyspark.sql.types import StringType

# Initialize SparkSession (already exists in Databricks notebooks)
spark = SparkSession.builder.appName("SpaceX Data Pipeline").getOrCreate()

# Initialize logging
log_data = []

def log_message(message, level="INFO"):
    """Log messages with timestamp and level"""
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    log_entry = f"[{timestamp}] [{level}] {message}"
    print(log_entry)
    log_data.append(log_entry)

def measure_execution_time(func):
    """Decorator to measure execution time of functions"""
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        log_message(f"Execution of {func.__name__} took {execution_time:.2f} seconds")
        return result
    return wrapper

@measure_execution_time
def fetch_data(url, data_name):
    """Fetch data from URL and return as JSON"""
    try:
        log_message(f"Fetching {data_name} data from {url}")
        response = requests.get(url)
        response.raise_for_status()  # Raise exception for bad status codes
        data = response.json()
        log_message(f"Successfully fetched {len(data)} {data_name} records")
        return data
    except requests.exceptions.RequestException as e:
        log_message(f"Error fetching {data_name} data: {str(e)}", "ERROR")
        raise

@measure_execution_time
def convert_to_dataframe(data, data_name):
    """Convert JSON data to Spark DataFrame using pandas as intermediate step"""
    try:
        log_message(f"Converting {data_name} data to DataFrame")
        
        # First convert to pandas DataFrame (which handles nested JSON well)
        pdf = pd.DataFrame(data)
        
        # For launches data, we need to handle the nested structures
        if data_name == "launches":
            # Extract rocket_id from the nested structure and add as a separate column
            pdf['rocket_id'] = pdf['rocket'].apply(lambda x: x.get('rocket_id') if isinstance(x, dict) else None)
            
            # Convert complex nested structures to strings to avoid schema inference issues
            for col_name in pdf.columns:
                if pdf[col_name].apply(lambda x: isinstance(x, (dict, list))).any():
                    pdf[col_name] = pdf[col_name].apply(lambda x: json.dumps(x) if x is not None else None)
        
        # Convert pandas DataFrame to Spark DataFrame
        df = spark.createDataFrame(pdf)
        
        log_message(f"Successfully created DataFrame with {df.count()} rows and {len(df.columns)} columns")
        return df
    except Exception as e:
        log_message(f"Error converting {data_name} data to DataFrame: {str(e)}", "ERROR")
        raise

@measure_execution_time
def join_data(launches_df, rockets_df):
    """Join launches with corresponding rocket data"""
    try:
        log_message("Joining launch data with rocket names")
        
        # Select only needed columns from rockets dataframe
        rockets_slim = rockets_df.select(col("rocket_id"), col("rocket_name"))
        
        # Join launches with rockets on rocket_id
        joined_df = launches_df.join(
            rockets_slim, 
            launches_df.rocket_id == rockets_slim.rocket_id, 
            "left"
        )
        
        # Log success details
        row_count = joined_df.count()
        log_message(f"Successfully joined data, resulting in {row_count} records")
        
        return joined_df
    except Exception as e:
        log_message(f"Error joining data: {str(e)}", "ERROR")
        raise

@measure_execution_time
def send_data_to_endpoint(data, url):
    """Send processed data to the specified endpoint"""
    try:
        log_message(f"Sending data to {url}")
        
        # Convert DataFrame to JSON format
        # Using toPandas() to get the data back to driver node - be careful with large datasets
        pd_df = data.toPandas()
        json_data = pd_df.to_json(orient="records")
        payload = {"data": json.loads(json_data)}
        
        # Send POST request to endpoint
        response = requests.post(url, json=payload)
        response.raise_for_status()
        
        # Log response details
        log_message(f"Successfully sent data. Response status: {response.status_code}")
        log_message(f"Response content: {response.text[:200]}...")
        
        return response
    except requests.exceptions.RequestException as e:
        log_message(f"Error sending data: {str(e)}", "ERROR")
        raise

def main():
    """Main pipeline function"""
    total_start_time = time.time()
    
    log_message("Starting SpaceX data pipeline")
    
    try:
        # 1. Fetch data from APIs
        launches_data = fetch_data("https://api.spacexdata.com/v3/launches", "launches")
        rockets_data = fetch_data("https://api.spacexdata.com/v3/rockets", "rockets")
        
        # 2. Convert to DataFrames
        launches_df = convert_to_dataframe(launches_data, "launches")
        rockets_df = convert_to_dataframe(rockets_data, "rockets")
        
        # 3. Print schema to help debugging
        log_message("Launches DataFrame Schema:")
        launches_df.printSchema()
        
        log_message("Rockets DataFrame Schema:")
        rockets_df.printSchema()
        
        # 4. Join data
        result_df = join_data(launches_df, rockets_df)
        
        # 5. Show sample of processed data
        log_message("Sample of processed data:")
        result_df.select("flight_number", "mission_name", "rocket_name").show(5)
        
        # 6. Send data to endpoint
        response = send_data_to_endpoint(result_df, "https://httpbin.org/post")
        
        # Pipeline completed successfully
        total_execution_time = time.time() - total_start_time
        log_message(f"Pipeline completed successfully in {total_execution_time:.2f} seconds")
        
    except Exception as e:
        log_message(f"Pipeline failed: {str(e)}", "ERROR")
        raise
    finally:
        # Save logs to DataFrame
        log_df = spark.createDataFrame([(log,) for log in log_data], ["log_message"])
        
        # Write logs to file - using Delta format which is better supported in Databricks
        log_df.write.format("delta").mode("overwrite").save("/pipeline_logs/spacex_pipeline_logs")

if __name__ == "__main__":
    main()

[2025-04-15 12:55:50] [ERROR] Pipeline failed: DataFrame columns must be unique for orient='records'.

In [0]:
# Databricks SpaceX Data Processing Pipeline
# This pipeline fetches SpaceX launch and rocket data, joins them, and posts results to an API endpoint.
# Compatible with Spark Connect / Databricks Serverless

import requests
import json
import time
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize SparkSession (already exists in Databricks notebooks)
spark = SparkSession.builder.appName("SpaceX Data Pipeline").getOrCreate()

# Initialize logging
log_data = []

def log_message(message, level="INFO"):
    """Log messages with timestamp and level"""
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    log_entry = f"[{timestamp}] [{level}] {message}"
    print(log_entry)
    log_data.append(log_entry)

def measure_execution_time(func):
    """Decorator to measure execution time of functions"""
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        log_message(f"Execution of {func.__name__} took {execution_time:.2f} seconds")
        return result
    return wrapper

@measure_execution_time
def fetch_data(url, data_name):
    """Fetch data from URL and return as JSON"""
    try:
        log_message(f"Fetching {data_name} data from {url}")
        response = requests.get(url)
        response.raise_for_status()  # Raise exception for bad status codes
        data = response.json()
        log_message(f"Successfully fetched {len(data)} {data_name} records")
        return data
    except requests.exceptions.RequestException as e:
        log_message(f"Error fetching {data_name} data: {str(e)}", "ERROR")
        raise

@measure_execution_time
def convert_to_dataframe(data, data_name):
    """Convert JSON data to Spark DataFrame using pandas as intermediate step"""
    try:
        log_message(f"Converting {data_name} data to DataFrame")
        
        # First convert to pandas DataFrame (which handles nested JSON well)
        pdf = pd.DataFrame(data)
        
        # For launches data, we need to handle the nested structures
        if data_name == "launches":
            # Extract rocket_id from the nested structure and add as a separate column
            pdf['rocket_id'] = pdf['rocket'].apply(lambda x: x.get('rocket_id') if isinstance(x, dict) else None)
            
            # Convert complex nested structures to strings to avoid schema inference issues
            for col_name in pdf.columns:
                if pdf[col_name].apply(lambda x: isinstance(x, (dict, list))).any():
                    pdf[col_name] = pdf[col_name].apply(lambda x: json.dumps(x) if x is not None else None)
        
        # Convert pandas DataFrame to Spark DataFrame
        df = spark.createDataFrame(pdf)
        
        log_message(f"Successfully created DataFrame with {df.count()} rows and {len(df.columns)} columns")
        return df
    except Exception as e:
        log_message(f"Error converting {data_name} data to DataFrame: {str(e)}", "ERROR")
        raise

@measure_execution_time
def join_data(launches_df, rockets_df):
    """Join launches with corresponding rocket data"""
    try:
        log_message("Joining launch data with rocket names")
        
        # Select only needed columns from rockets dataframe and rename rocket_id to avoid duplicate columns
        rockets_slim = rockets_df.select(
            col("rocket_id").alias("r_rocket_id"), 
            col("rocket_name")
        )
        
        # Join launches with rockets on rocket_id
        joined_df = launches_df.join(
            rockets_slim, 
            launches_df.rocket_id == rockets_slim.r_rocket_id, 
            "left"
        )
        
        # Log success details
        row_count = joined_df.count()
        log_message(f"Successfully joined data, resulting in {row_count} records")
        
        return joined_df
    except Exception as e:
        log_message(f"Error joining data: {str(e)}", "ERROR")
        raise

@measure_execution_time
def send_data_to_endpoint(data, url):
    """Send processed data to the specified endpoint"""
    try:
        log_message(f"Sending data to {url}")
        
        # Select key columns to reduce data size and ensure column uniqueness
        # Drop duplicate columns and select only what we need
        slim_df = data.select(
            "flight_number", 
            "mission_name", 
            "launch_year", 
            "launch_date_utc", 
            "launch_success", 
            "details", 
            "rocket_id",  # from launches
            "rocket_name"  # from rockets
        )
        
        # Convert DataFrame to pandas for JSON serialization
        pd_df = slim_df.toPandas()
        
        # Check for duplicate column names before conversion
        if pd_df.columns.duplicated().any():
            # Get list of duplicate columns
            dupes = pd_df.columns[pd_df.columns.duplicated()].tolist()
            log_message(f"Found duplicate column names: {dupes}", "WARNING")
            
            # Rename duplicated columns
            for i, col_name in enumerate(pd_df.columns):
                if col_name in dupes:
                    pd_df.rename(columns={col_name: f"{col_name}_{i}"}, inplace=True)
        
        # Convert to JSON for API request
        json_data = pd_df.to_json(orient="records")
        payload = {"data": json.loads(json_data)}
        
        # Send POST request to endpoint
        response = requests.post(url, json=payload)
        response.raise_for_status()
        
        # Log response details
        log_message(f"Successfully sent data. Response status: {response.status_code}")
        log_message(f"Response content: {response.text[:200]}...")
        
        return response
    except Exception as e:
        log_message(f"Error sending data: {str(e)}", "ERROR")
        raise

def main():
    """Main pipeline function"""
    total_start_time = time.time()
    
    log_message("Starting SpaceX data pipeline")
    
    try:
        # 1. Fetch data from APIs
        launches_data = fetch_data("https://api.spacexdata.com/v3/launches", "launches")
        rockets_data = fetch_data("https://api.spacexdata.com/v3/rockets", "rockets")
        
        # 2. Convert to DataFrames
        launches_df = convert_to_dataframe(launches_data, "launches")
        rockets_df = convert_to_dataframe(rockets_data, "rockets")
        
        # 3. Print schema to help debugging
        log_message("Launches DataFrame Schema:")
        launches_df.printSchema()
        
        log_message("Rockets DataFrame Schema:")
        rockets_df.printSchema()
        
        # 4. Join data
        result_df = join_data(launches_df, rockets_df)
        
        # 5. Show sample of processed data
        log_message("Sample of processed data:")
        result_df.select("flight_number", "mission_name", "rocket_name").show(5)
        
        # 6. Send data to endpoint
        response = send_data_to_endpoint(result_df, "https://httpbin.org/post")
        
        # Pipeline completed successfully
        total_execution_time = time.time() - total_start_time
        log_message(f"Pipeline completed successfully in {total_execution_time:.2f} seconds")
        
    except Exception as e:
        log_message(f"Pipeline failed: {str(e)}", "ERROR")
        raise
    finally:
        # Save logs to DataFrame
        log_df = spark.createDataFrame([(log,) for log in log_data], ["log_message"])
        
        # Write logs to file - using Delta format which is better supported in Databricks
        log_df.write.format("delta").mode("overwrite").save("/pipeline_logs/spacex_pipeline_logs")

if __name__ == "__main__":
    main()